In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split, TimeSeriesSplit, StratifiedKFold, StratifiedGroupKFold, GroupShuffleSplit
from sklearn.metrics import roc_auc_score
import random
from statistics import mean
from tqdm.auto import tqdm, trange
from sklearn.base import BaseEstimator

In [2]:
train_data = pd.read_csv('dodohack/Data Secrets First Cup/train_target.csv',parse_dates=['LocalBeginDate','LocalEndDate'])
test_data = pd.read_csv('dodohack/Data Secrets First Cup/test.csv',parse_dates=['LocalBeginDate','LocalEndDate'])
sample_submit = pd.read_csv('dodohack/Data Secrets First Cup/submit.csv')
orders_data = pd.read_csv('dodohack/Data Secrets First Cup/orders.csv',parse_dates=['SaleDate','Date'])
mobile_data = pd.read_csv('dodohack/Data Secrets First Cup/mobile_events.csv',parse_dates=['Timestamp'])
clients_data = pd.read_csv('dodohack/Data Secrets First Cup/clients_promo_october.csv',parse_dates=['LocalBeginDate','LocalEndDate'])

In [3]:
def mounth_count_day(x):
    months = [31,28,31,30,31,30,31,31,30,31,30,31]
    return sum(months[:x])

def create_time_features(df,time_col):
    df['month'] = df[time_col].dt.month
    df['day'] = df[time_col].dt.day
    df['hour'] = df[time_col].dt.hour
    df['year'] = df[time_col].dt.year
    df['weekofyear'] = df[time_col].apply(lambda x: x.weekofyear)
    df['dayofweek'] = df[time_col].dt.dayofweek
    df['dayofyear'] = df[time_col].dt.dayofyear
    df['absolute_time'] = (df[time_col] - df[time_col].min()).dt.days
    
    df['all_day_time'] = df[time_col].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    df['all_week_time'] = df[time_col].apply(lambda x: x.dayofweek * 24 + x.hour)
    
    df['cl_early_morning'] = ((df['hour'] > 4) & (df['hour'] <= 8)).astype('int16')
    df['cl_is_weekend'] = (df['dayofweek'] > 4).astype('int16')
    return df

In [4]:
mobile_data = create_time_features(mobile_data,'Timestamp')
orders_data = create_time_features(orders_data,'SaleDate')

In [5]:
mobile_data

,ClientUUId,VisitToken,EventName,Platform,Timestamp,month,day,hour,year,weekofyear,dayofweek,dayofyear,absolute_time,all_day_time,all_week_time,cl_early_morning,cl_is_weekend
0,000D3A22FA54A81611EB315CF5443815,976627AD-76DC-41D9-981E-F2A6CA14B3A9,screen_menu,ios,2023-10-29 16:14:21.343000+00:00,10,29,16,2023,43,6,302,387,58461,160,0,1
1,2A4A64AF719E994511EE3F2D8B31E188,5b22e610-c71e-4b6a-a842-71e8575ece87,screen_menu,android,2023-10-29 11:04:36.484000+00:00,10,29,11,2023,43,6,302,386,39876,155,0,1
2,000D3AAC977BBB2F11ECDD081F5481AE,BDCEB72F-2B6A-4565-801A-6CB729279C6F,open_app,ios,2023-10-29 11:14:34.723000+00:00,10,29,11,2023,43,6,302,386,40474,155,0,1
3,000D3A21DA51A81411EAE468856F96F1,A66E2DA6-4E2E-4C37-9FBF-627732623212,add_to_cart,ios,2023-10-29 14:03:08.363000+00:00,10,29,14,2023,43,6,302,387,50588,158,0,1
4,000D3A23B0DC80D811E67F4D0A9CA345,1BE8139F-A3D3-439F-A51B-5CBD4D3FFF1B,screen_menu,ios,2023-10-29 08:35:40.127000+00:00,10,29,8,2023,43,6,302,386,30940,152,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3916649,000D3A25D54580E011E707D4762E0C84,C5EC0144-6C3C-45C2-B0A4-A7A2B6B45444,open_app,ios,2023-10-14 06:00:21.558000+00:00,10,14,6,2023,41,5,287,371,21621,126,1,1
3916650,CEA31E584572ADD411ED0E4D10ED070E,5a8b6b6c-e4ad-4fd9-aad9-895fa987ddf1,screen_profile,android,2023-10-13 12:55:57.752000+00:00,10,13,12,2023,41,4,286,370,46557,108,0,0
3916651,000D3A25D54580E011E707D4762E0C84,C5EC0144-6C3C-45C2-B0A4-A7A2B6B45444,close_app,ios,2023-10-14 06:00:21.515000+00:00,10,14,6,2023,41,5,287,371,21621,126,1,1
3916652,CEA31E584572ADD411ED0E4D10ED070E,5a8b6b6c-e4ad-4fd9-aad9-895fa987ddf1,screen_menu,android,2023-10-13 12:55:56.303000+00:00,10,13,12,2023,41,4,286,370,46556,108,0,0


In [6]:
def get_shift_features(data,time_col='absolute_time',user_col='ClientUUId',prefix=''):
    shift_time = data.groupby(user_col)[time_col].agg(lambda x: np.diff(np.sort(x)).tolist())
    shift_features = pd.DataFrame(index=shift_time.index)
    shift_features[f'{prefix}_mean'] = shift_time.apply(np.mean)
    shift_features[f'{prefix}_min'] = shift_time.apply(np.min)
    shift_features[f'{prefix}_max'] = shift_time.apply(np.max)
    shift_features[f'{prefix}_std'] = shift_time.apply(np.std)
    shift_features[f'{prefix}_polyfit'] = shift_time.apply(lambda x:np.polyfit(range(len(x)), x, 1)[1] if len(x) > 10 else -100)
    shift_features[f'{prefix}_polyfit_st'] = shift_time.apply(lambda x:np.polyfit(range(len(x)), x, 1)[0] if len(x) > 10 else -100)
    return shift_features


def get_mobile_data_features(mobile_data):
    time_day_feats = mobile_data.groupby('ClientUUId')['absolute_time'].agg(['min','max','mean','std','nunique','count'])
    time_day_feats['diff'] = time_day_feats['max'] - time_day_feats['min']
    time_day_feats['otn_diff'] = time_day_feats['diff'] / time_day_feats['mean']
    time_day_feats['designity'] = time_day_feats['nunique'] / time_day_feats['count']
    time_day_feats.columns = [f'mobile_time_{x}' for x in time_day_feats.columns]
    
    time_shift_feats = get_shift_features(mobile_data,prefix='mobile_data')
    
    time_hour_features = mobile_data.groupby('ClientUUId')['hour'].agg(['min','max','mean','std','nunique'])
    time_hour_features.columns = [f'mobile_hour_{x}' for x in time_hour_features.columns]
    
    
    platform_featues = pd.pivot_table(
                             mobile_data,
                             index =['ClientUUId'],
                             values=['absolute_time'],
                             columns=['Platform'],
                             aggfunc=['count','nunique','mean','min','max','std']
    ).fillna(-1).sort_index()
    
    event_featues = pd.pivot_table(
                             mobile_data,
                             index =['ClientUUId'],
                             values=['absolute_time'],
                             columns=['EventName'],
                             aggfunc=['count','nunique','mean','min','max','std']
    ).fillna(-1).sort_index()
    
    day_of_week_featues = pd.pivot_table(
                             mobile_data,
                             index =['ClientUUId'],
                             values=['dayofweek'],
                             columns=['EventName'],
                             aggfunc=['count','nunique','mean']
    ).fillna(-1).sort_index()

    
    hour_counters = pd.pivot_table(
                             mobile_data,
                             index =['ClientUUId'],
                             values=['absolute_time'],
                             columns=['hour'],
                             aggfunc=['count','nunique','mean']
    ).fillna(-1).sort_index()
    
    platform_featues.columns = [f'mobile_platform_{x[0]}_{x[2]}' for x in platform_featues.columns]
    event_featues.columns = [f'mobile_event_{x[0]}_{x[2]}' for x in event_featues.columns]
    hour_counters.columns = [f'mobile_hour_{x[0]}_{x[2]}' for x in hour_counters.columns]
    day_of_week_featues.columns = [f'mobile_dayofweek_{x[0]}_{x[2]}' for x in day_of_week_featues.columns]
    
    mobile_data['VisitToken_count'] = mobile_data.groupby('VisitToken')['EventName'].transform('count')
    visitors_features = mobile_data.groupby('ClientUUId')['VisitToken_count'].agg(['min','max','mean','std'])
    visitors_features.columns = [f'mobile_visitors_{x}' for x in visitors_features.columns]
    visitors_features['mobile_visitors_nunique'] = mobile_data.groupby('ClientUUId')['VisitToken'].agg('nunique')
    
    for col in event_featues.columns:
        event_featues[f'{col}_otn'] = event_featues[col] / time_day_feats['mobile_time_count']
    for col in platform_featues.columns:
        platform_featues[f'{col}_otn'] = platform_featues[col] / time_day_feats['mobile_time_count']
    
    return pd.concat([
        time_day_feats,
        time_hour_features,
        platform_featues,
        event_featues,
        hour_counters,
        visitors_features,
        time_shift_feats
    ],axis=1)
    

In [7]:
mobile_features = get_mobile_data_features(mobile_data)

In [8]:
def get_clients_features(clients_data):
    clients_data['vugoda'] = clients_data['Discount'] / clients_data['OrderPrice']
    clients_data['Id+OrderType'] = clients_data['Id'].astype(str) + clients_data['OrderType'].astype(str)
    
    order_price_feats = clients_data.groupby('ClientUUId')['OrderPrice'].agg(['mean','std','sum','mean','max','count'])
    Discount_feats = clients_data.groupby('ClientUUId')['Discount'].agg(['mean','std','sum','mean','max','nunique'])
    vugoda_feats = clients_data.groupby('ClientUUId')['vugoda'].agg(['mean','std','sum','mean','max','nunique'])
    
    order_price_feats.columns = [f'clients_order_price_{x}' for x in order_price_feats.columns]
    Discount_feats.columns = [f'clients_discount_{x}' for x in Discount_feats.columns]
    vugoda_feats.columns = [f'clients_vugoda_{x}' for x in vugoda_feats.columns]
    
    id_features = pd.pivot_table(
                             clients_data,
                             index =['ClientUUId'],
                             values=['OrderPrice',],
                             columns=['Id'],
                             aggfunc=['count','nunique','sum']
    ).fillna(-1).sort_index()
    
    order_features = pd.pivot_table(
                             clients_data,
                             index =['ClientUUId'],
                             values=['OrderPrice',],
                             columns=['OrderType'],
                             aggfunc=['count','nunique','sum']
    ).fillna(-1).sort_index()
    
    orderid_features = pd.pivot_table(
                             clients_data,
                             index =['ClientUUId'],
                             values=['OrderPrice'],
                             columns=['Id+OrderType'],
                             aggfunc=['count']
    ).fillna(-1).sort_index()
    
    id_features.columns = [f'clients_id_{x[0]}_{x[1]}_{x[2]}' for x in id_features.columns]
    order_features.columns = [f'clients_order_{x[0]}_{x[1]}_{x[2]}' for x in order_features.columns]
    orderid_features.columns = [f'clients_orderid_{x[0]}_{x[2]}' for x in orderid_features.columns]

    return pd.concat([
        id_features,
        order_features,
        orderid_features,
        order_price_feats,
        Discount_feats,
        vugoda_feats
    ],axis=1)

In [9]:
clients_features = get_clients_features(clients_data)

In [10]:
def get_order_data_feats(orders_data):
    time_day_feats = mobile_data.groupby('ClientUUId')['absolute_time'].agg(['min','max','mean','std','nunique','count'])
    time_day_feats['diff'] = time_day_feats['max'] - time_day_feats['min']
    time_day_feats['otn_diff'] = time_day_feats['diff'] / time_day_feats['mean']
    time_day_feats['designity'] = time_day_feats['nunique'] / time_day_feats['count']
    time_day_feats.columns = [f'order_time_{x}' for x in time_day_feats.columns]
    
    time_shift_feats = get_shift_features(orders_data,prefix='orders_data')
    
    orders_data = orders_data.fillna(0)
    orders_data['OrderUUId_count'] = orders_data.groupby('OrderUUId').transform('count')['NewClient']
    orders_data['addressId_count'] = orders_data.groupby('addressId').transform('count')['NewClient']
    orders_data['deliverySectorId_count'] = orders_data.groupby('deliverySectorId').transform('count')['NewClient']
    orders_data['UnitUUId_count'] = orders_data.groupby('UnitUUId').transform('count')['NewClient']
    orders_data['ProductUUId_count'] = orders_data.groupby('ProductUUId').transform('count')['NewClient']
    
    orders_data['OrderUUId_appl_mean'] = orders_data.groupby('OrderUUId')['apply_promo'].agg('mean').loc[orders_data['OrderUUId']].values
    orders_data['addressId_appl_mean'] = orders_data.groupby('addressId')['apply_promo'].agg('mean').loc[orders_data['addressId']].values
    orders_data['deliverySectorId_appl_mean'] = orders_data.groupby('deliverySectorId')['apply_promo'].agg('mean').loc[orders_data['deliverySectorId']].values
    orders_data['UnitUUId_appl_mean'] = orders_data.groupby('UnitUUId')['apply_promo'].agg('mean').loc[orders_data['UnitUUId']].values
    orders_data['ProductUUId_appl_mean'] = orders_data.groupby('ProductUUId')['apply_promo'].agg('mean').loc[orders_data['ProductUUId']].values
    
    orders_data['OrderUUId_appl_sum'] = orders_data.groupby('OrderUUId')['apply_promo'].agg('sum').loc[orders_data['OrderUUId']].values
    orders_data['addressId_appl_sum'] = orders_data.groupby('addressId')['apply_promo'].agg('sum').loc[orders_data['addressId']].values
    orders_data['deliverySectorId_appl_sum'] = orders_data.groupby('deliverySectorId')['apply_promo'].agg('sum').loc[orders_data['deliverySectorId']].values
    orders_data['UnitUUId_appl_sum'] = orders_data.groupby('UnitUUId')['apply_promo'].agg('sum').loc[orders_data['UnitUUId']].values
    orders_data['ProductUUId_appl_sum'] = orders_data.groupby('ProductUUId')['apply_promo'].agg('sum').loc[orders_data['ProductUUId']].values


    orders_data['Skidka'] = orders_data['MenuPrice'] - orders_data['ProductTotalPrice']
    orders_data['Skidka_otn'] = orders_data['Skidka'] / orders_data['ProductTotalPrice']
    
    appl_features = orders_data.groupby('ClientUUId')['apply_promo'].agg(['mean','count','std','sum'])
    newcl_features = orders_data.groupby('ClientUUId')['NewClient'].agg(['sum'])
    appl_features.columns = [f'order_appl_{x}' for x in appl_features.columns]
    newcl_features.columns = [f'order_newcl_{x}' for x in newcl_features.columns]
    
    category_features = pd.pivot_table(
                             orders_data,
                             index =['ClientUUId'],
                             values=['MenuPrice','absolute_time'],
                             columns=['apply_promo','CategoryId'],
                             aggfunc=['count','sum','mean','std']
    ).fillna(-1).sort_index()
    
    payment_features = pd.pivot_table(
                             orders_data,
                             index =['ClientUUId'],
                             values=['MenuPrice','absolute_time'],
                             columns=['apply_promo','OrderPaymentType'],
                             aggfunc=['count','nunique','sum','mean','std']
    ).fillna(-1).sort_index()

    
    order_features = pd.pivot_table(
                             orders_data,
                             index =['ClientUUId'],
                             values=['MenuPrice','absolute_time'],
                             columns=['apply_promo','OrderType'],
                             aggfunc=['count','nunique','sum','mean','std']
    ).fillna(-1).sort_index()
    
    order_state_features = pd.pivot_table(
                             orders_data,
                             index =['ClientUUId'],
                             values=['MenuPrice'],
                             columns=['apply_promo','OrderState'],
                             aggfunc=['count','nunique','sum','mean']
    ).fillna(-1).sort_index()


    features = pd.pivot_table(
                             orders_data,
                             index =['ClientUUId'],
                             values=['MenuPrice','ClientOrderNumber',
                                     'ProductTotalPrice','OrderTotalPrice',
                                     'OrderUUId_count','Skidka','Skidka_otn',
                                     'addressId_count','deliverySectorId_count',
                                     'UnitUUId_count','ProductUUId_count',
                                     'OrderUUId_appl_mean','addressId_appl_mean',
                                     'deliverySectorId_appl_mean',
                                     'UnitUUId_appl_mean','ProductUUId_appl_mean',
                                     'OrderUUId_appl_sum','addressId_appl_sum',
                                     'deliverySectorId_appl_sum',
                                     'UnitUUId_appl_sum','ProductUUId_appl_sum',
                                     'hour','absolute_time','dayofweek','month','all_day_time'],
                             columns=['apply_promo'],
                             aggfunc=['mean','sum','min','max','std','nunique',lambda x:np.max(x) - np.min(x)]
        
    ).fillna(-1).sort_index()
    
    features.columns = [f'order_feats_{x[0]}_{x[1]}_{x[2]}' for x in features.columns]
    payment_features.columns = [f'order_payment_{x[0]}_{x[1]}_{x[2]}_{x[3]}' for x in payment_features.columns]
    order_features.columns = [f'order_order_{x[0]}_{x[1]}_{x[2]}_{x[3]}' for x in order_features.columns]
    category_features.columns = [f'order_order_{x[0]}_{x[1]}_{x[2]}_{x[3]}' for x in category_features.columns]
    order_state_features.columns = [f'order_order_{x[0]}_{x[1]}_{x[2]}_{x[3]}' for x in order_state_features.columns]
    
    return pd.concat([
        appl_features,
        newcl_features,
        payment_features,
        order_features,
        time_day_feats,
        order_state_features,
        time_shift_feats,
        features,
    ],axis=1)

In [11]:
order_features = get_order_data_feats(orders_data)

/tmp/ipykernel_168/1861438450.py:88: RuntimeWarning: invalid value encountered in scalar subtract
  aggfunc=['mean','sum','min','max','std','nunique',lambda x:np.max(x) - np.min(x)]


In [12]:
mobile_embeds = pd.read_parquet('mobile_embeds.parquet')
order_embeds = pd.read_parquet('order_embeds.parquet')

In [13]:
mobile_embeds.columns = [f"mobile_{x}" for x in mobile_embeds.columns]
order_embeds.columns = [f"order_{x}" for x in order_embeds.columns]

In [14]:
all_features = pd.concat([
    order_features,
    clients_features,
    mobile_features,
    mobile_embeds,
    order_embeds
],axis=1).fillna(-100)

In [15]:
all_features = all_features.T.drop_duplicates().T
all_features['ClientUUId'] = all_features.index
all_features.index = range(len(all_features))

In [16]:
all_features.shape

(30000, 919)

In [17]:
train_data = train_data.merge(all_features)

In [18]:
params = {
    'iterations':1000,
    'learning_rate':0.01,
    'loss_function':'CrossEntropy',
    'max_depth':7,
    'eval_metric':'AUC',
    'task_type':'GPU',
    'random_seed':56
}

drop_cols = [
    'LocalBeginDate',
    'LocalEndDate',
    'ClientUUId',
]

cat_cols = [
    'Id',
    'OrderType',
]

label_col = 'apply_promo'
algorithm = 'RecursiveByShapValues'
num_fold = 5
n_fold_test = 10
test_c_stop = 10
num_repits = 3
num_features_to_select = 128
features_for_select = '0-919'
steps = 10

In [19]:
def model_builder(train_data):
    splitter = GroupShuffleSplit(test_size=0.2, n_splits=2,random_state=56)
    split = splitter.split(train_data, groups=train_data['ClientUUId'])
    train_inds, test_inds = next(split)

    train_df, eval_df = train_data.iloc[train_inds], train_data.iloc[test_inds]
    
    train_pool = Pool(
        train_df.drop([label_col]+drop_cols,axis=1),
        label = train_df[label_col],
        cat_features = cat_cols
    )

    eval_pool = Pool(
        eval_df.drop([label_col]+drop_cols,axis=1),
        label = eval_df[label_col],
        cat_features = cat_cols
    )
    
    cbm = CatBoostClassifier(**params)
    out = cbm.select_features(
        train_pool,
        eval_set=eval_pool,
        algorithm=algorithm,
        features_for_select=features_for_select,
        num_features_to_select=num_features_to_select,
        steps=steps,
        shap_calc_type=None,
        train_final_model=True,
        verbose=300,
        plot=False
    )
    return cbm, out

In [20]:
class TestKFoldWrapper():
    def __init__(self,num_folds=10,top_c=1,random_state=56):
        self.top_c = top_c
        self.kfold = StratifiedGroupKFold(num_folds,random_state=random_state,shuffle=True)
    
    def run_experiments(self,model_builder,train_data,label_col=None):
        self.models = []
        self.outs = []
        c = 0
        
        for train_index, test_index in tqdm(self.kfold.split(train_data,train_data[label_col],groups=train_data['ClientUUId'])):
            train_df = train_data.iloc[train_index]
            test_df = train_data.iloc[test_index]
            
            model,outs = model_builder(train_df)
            self.outs.append(outs)
            self.models.append(model)
            c += 1
            if c >= self.top_c:
                break

In [21]:
evaluator = TestKFoldWrapper(n_fold_test,test_c_stop)

evaluator.run_experiments(
    model_builder=model_builder,
    train_data=train_data,
    label_col=label_col
)

0it [00:00, ?it/s]

Default metric period is 5 because AUC is/are not implemented for GPU


Step #1 out of 10
0:	test: 0.7365037	best: 0.7365037 (0)	total: 81.7ms	remaining: 1m 21s
300:	test: 0.7682055	best: 0.7752452 (165)	total: 24.4s	remaining: 56.7s
600:	test: 0.7637926	best: 0.7752452 (165)	total: 49.4s	remaining: 32.8s
900:	test: 0.7583201	best: 0.7752452 (165)	total: 1m 14s	remaining: 8.15s
999:	test: 0.7577667	best: 0.7752452 (165)	total: 1m 22s	remaining: 0us
bestTest = 0.7752452493
bestIteration = 165
Shrink model to first 166 iterations.
Feature #412 eliminated
Feature #859 eliminated
Feature #918 eliminated
Feature #337 eliminated
Feature #60 eliminated
Feature #901 eliminated
Feature #416 eliminated
Feature #178 eliminated
Feature #88 eliminated
Feature #332 eliminated
Feature #329 eliminated
Feature #334 eliminated
Feature #347 eliminated
Feature #163 eliminated
Feature #310 eliminated
Feature #75 eliminated
Feature #405 eliminated
Feature #331 eliminated
Feature #328 eliminated
Feature #149 eliminated
Feature #31 eliminated
Feature #872 eliminated
Feature #427 

Default metric period is 5 because AUC is/are not implemented for GPU


Step #1 out of 10
0:	test: 0.6603118	best: 0.6603118 (0)	total: 81.2ms	remaining: 1m 21s
300:	test: 0.7431909	best: 0.7456450 (285)	total: 24.7s	remaining: 57.3s
600:	test: 0.7474186	best: 0.7485344 (510)	total: 49.9s	remaining: 33.1s
900:	test: 0.7491856	best: 0.7495228 (815)	total: 1m 14s	remaining: 8.21s
999:	test: 0.7487842	best: 0.7495228 (815)	total: 1m 23s	remaining: 0us
bestTest = 0.7495228052
bestIteration = 815
Shrink model to first 816 iterations.
Feature #620 eliminated
Feature #79 eliminated
Feature #78 eliminated
Feature #109 eliminated
Feature #24 eliminated
Feature #275 eliminated
Feature #910 eliminated
Feature #76 eliminated
Feature #376 eliminated
Feature #914 eliminated
Feature #66 eliminated
Feature #399 eliminated
Feature #90 eliminated
Feature #64 eliminated
Feature #25 eliminated
Feature #603 eliminated
Feature #62 eliminated
Feature #286 eliminated
Feature #757 eliminated
Feature #68 eliminated
Feature #867 eliminated
Feature #692 eliminated
Feature #435 elimin

Default metric period is 5 because AUC is/are not implemented for GPU


Step #1 out of 10
0:	test: 0.6630685	best: 0.6630685 (0)	total: 89.2ms	remaining: 1m 29s
300:	test: 0.7574582	best: 0.7574582 (300)	total: 29.8s	remaining: 1m 9s
600:	test: 0.7682198	best: 0.7682741 (595)	total: 58.8s	remaining: 39.1s
900:	test: 0.7730065	best: 0.7734133 (840)	total: 1m 25s	remaining: 9.39s
999:	test: 0.7729886	best: 0.7739138 (965)	total: 1m 34s	remaining: 0us
bestTest = 0.7739138007
bestIteration = 965
Shrink model to first 966 iterations.
Feature #86 eliminated
Feature #755 eliminated
Feature #859 eliminated
Feature #286 eliminated
Feature #109 eliminated
Feature #912 eliminated
Feature #218 eliminated
Feature #142 eliminated
Feature #713 eliminated
Feature #917 eliminated
Feature #25 eliminated
Feature #92 eliminated
Feature #24 eliminated
Feature #884 eliminated
Feature #508 eliminated
Feature #514 eliminated
Feature #147 eliminated
Feature #220 eliminated
Feature #699 eliminated
Feature #853 eliminated
Feature #516 eliminated
Feature #906 eliminated
Feature #37 e

Default metric period is 5 because AUC is/are not implemented for GPU


Step #1 out of 10
0:	test: 0.7205787	best: 0.7205787 (0)	total: 80.5ms	remaining: 1m 20s
300:	test: 0.7837433	best: 0.7837433 (300)	total: 24.7s	remaining: 57.3s
600:	test: 0.7842496	best: 0.7858923 (435)	total: 49.7s	remaining: 33s
900:	test: 0.7850044	best: 0.7858923 (435)	total: 1m 14s	remaining: 8.19s
999:	test: 0.7857579	best: 0.7861707 (965)	total: 1m 22s	remaining: 0us
bestTest = 0.7861706614
bestIteration = 965
Shrink model to first 966 iterations.
Feature #212 eliminated
Feature #306 eliminated
Feature #723 eliminated
Feature #203 eliminated
Feature #411 eliminated
Feature #25 eliminated
Feature #865 eliminated
Feature #755 eliminated
Feature #79 eliminated
Feature #283 eliminated
Feature #868 eliminated
Feature #109 eliminated
Feature #31 eliminated
Feature #548 eliminated
Feature #95 eliminated
Feature #910 eliminated
Feature #884 eliminated
Feature #303 eliminated
Feature #812 eliminated
Feature #832 eliminated
Feature #246 eliminated
Feature #341 eliminated
Feature #442 el

KeyboardInterrupt: 

In [23]:
top_k_feats = set(evaluator.outs[0]['selected_features_names']) & set(evaluator.outs[1]['selected_features_names']) & set(evaluator.outs[2]['selected_features_names'])

In [30]:
np.save('best_features6_2.npy',evaluator.outs[2]['selected_features_names'])

In [31]:
evaluator.models[0].get_feature_importance(prettified=True)[:60]

,Feature Id,Importances
0,mobile_event_nunique_apply_personal_offer,5.392954
1,mobile_event_nunique_screen_profile,4.570807
2,"clients_orderid_count_71,2,3",3.418077
3,mobile_event_max_apply_personal_offer_otn,3.127639
4,order_feats_nunique_all_day_time_1,3.093048
5,order_feats_nunique_ClientOrderNumber_1,2.743967
6,order_feats_std_ClientOrderNumber_1,2.465097
7,mobile_event_nunique_apply_personal_offer_otn,2.048009
8,mobile_event_mean_apply_personal_offer,1.980343
9,mobile_event_std_apply_personal_offer_otn,1.975341


In [27]:
model = model_builder(train_data)

Default metric period is 5 because AUC is/are not implemented for GPU


Step #1 out of 10
0:	test: 0.6805359	best: 0.6805359 (0)	total: 50.1ms	remaining: 50s


KeyboardInterrupt: 

In [33]:
test_data = test_data.merge(all_features)

In [48]:
preds = evaluator.models[0].predict(test_data[],drop_cols=drop_cols,cat_features=cat_cols)

In [49]:
sample_submit['apply_promo'] = preds 

In [50]:
sample_submit.to_csv('./submits/sub_dodo8.csv',index=False)

In [40]:
simple_sub = pd.read_csv('submits/sub_dodo15.csv')
simple_sub['apply_promo'] = pd.read_csv('submits/sub_dodo12.csv')['apply_promo'].values * 0.05 + simple_sub['apply_promo'].values * 0.9

In [41]:
simple_sub.to_csv('./submits/ens_dodo_3.csv',index=False)

In [42]:
simple_sub

,apply_promo
0,0.030221
1,0.096121
2,0.024879
3,0.030460
4,0.039861
...,...
6801,0.013490
6802,0.017919
6803,0.031535
6804,0.075184
